In [1]:
import pandas as pd
from gensim import corpora, models
from gensim.corpora import Dictionary
from scipy.sparse import csr_matrix
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import keras as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import tensorflow as tf
import pymongo

import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
import string, re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate,MaxPooling1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
import keras as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
embed_size = 300 # how big is each word vector
max_features = 200000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 300 

In [3]:
client = pymongo.MongoClient(host="192.168.0.106", port=29197)
client["github"].authenticate("github", "git332", "github")
db = client["github"]

In [4]:
lang_set = [x["name"] for x in db.project_lang_30.find()]
npm_tag_set = ["npm_"+x["name"] for x in db.npm_tag_stem_top100.find()]
pkg_tag_set = ["pkg_"+x["name"] for x in db.composer_tag_stem_top100.find()]
pypi_tag_set = ["pypi_"+x["name"] for x in db.pypi_tag_stem_top100.find()]

In [5]:
train_col = ["file_npm", "file_pypi", "file_composer"]
train_col.extend(lang_set)
train_col.extend(npm_tag_set)
train_col.extend(pkg_tag_set)
train_col.extend(pypi_tag_set)
len(train_col)

333

In [6]:
tag_set = [x["name"] for x in db.project_tag_more_than_100.find()]
len(tag_set)

116

In [79]:
lda = models.ldamodel.LdaModel.load('../model/readme_lda_256.model')

In [7]:
data_raw = pd.read_csv("../data/data_raw.csv", index_col=0)
data_raw.head(4)

,project_link,file_readme,file_npm,file_pypi,file_composer,JavaScript,HTML,CSS,Python,Shell,...,pytorch,cach,laravel,data-sci,natural-language-process,authent,computer-vis,compos,python3,yii2
0,/freeCodeCamp/freeCodeCamp,welcom freecodecamp org s open sourc codebas c...,1,0,0,0.941868,0.017145,0.039971,0.000000,0.001017,...,0,0,0,0,0,0,0,0,0,0
1,/vuejs/vue,support vue jsvue js mit licens open sourc pro...,1,0,0,0.976735,0.006227,0.003945,0.000000,0.001170,...,0,0,0,0,0,0,0,0,0,0
2,/twbs/bootstrap,bootstrap sleek intuit power end framework fas...,1,0,0,0.494735,0.182466,0.320497,0.000000,0.001512,...,0,0,0,0,0,0,0,0,0,0
3,/facebook/react,react middot react javascript librari build us...,1,0,0,0.951378,0.017036,0.003504,0.000079,0.001854,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data_tag = np.array(data_raw[tag_set]).astype(np.int64)

In [9]:
readme_x = data_raw["file_readme"].fillna("_##_").values

In [10]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(readme_x))
readme_x_tz = tokenizer.texts_to_sequences(readme_x)

In [11]:
## Pad the sentences 
train_X_seq = pad_sequences(readme_x_tz, maxlen=maxlen)
train_X_seq.shape

(25467, 300)

In [12]:
vocab = tokenizer.word_index

In [13]:
data_minus_readme = data_raw[train_col]
data_minus_readme.shape

(25467, 333)

In [14]:
train_x, test_x, train_y, test_y = train_test_split(train_X_seq, data_tag, train_size=0.75, test_size=0.25, random_state=2019)

In [15]:
train_x.shape, test_x.shape

((19100, 300), (6367, 300))

In [16]:
# model define



# model = K.models.Sequential()
# model.add(Embedding(len(vocab), 100, input_length=300))
# model.add(Flatten())
# model.add(K.layers.Dense(units=200, input_dim=100, kernel_initializer=init, activation='relu'))
# model.add(K.layers.Dense(units=200, kernel_initializer=init, activation='relu'))
# model.add(K.layers.Dense(units=200, kernel_initializer=init, activation='relu'))
# model.add(K.layers.Dense(units=1, kernel_initializer=init, activation='relu'))
# model.compile(loss='mean_squared_error', optimizer=simple_adam, metrics=['categorical_accuracy'])

def BuildTextCNN(maxlen=200, max_features=20000, embed_size=64):
    comment_seq = Input(shape=[maxlen], name='x_seq')
    emb_comment = Embedding(max_features, embed_size)(comment_seq)
    convs = []
    filter_sizes = [2, 3, 4, 5]
    for fsz in filter_sizes:
        l_conv = Conv1D(filters=100, kernel_size=fsz, activation='relu')(emb_comment)
        l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    merge = concatenate(convs, axis=1)
    out = Dropout(0.5)(merge)
    output = Dense(200, activation='relu')(out)
    output = Dense(units=116, activation='relu')(output)
    model = Model([comment_seq], output)
    return model

In [18]:
init = K.initializers.glorot_uniform(seed=1)
simple_adam = K.optimizers.Adam(lr=0.0001)
textcnn = BuildTextCNN(maxlen=maxlen,max_features=len(vocab))
textcnn.compile(loss='mean_squared_error', optimizer=simple_adam, metrics=['categorical_accuracy'])

In [19]:
b_size = 1024
max_epochs = 20
print("Starting training ")
h = textcnn.fit(train_x, train_y, batch_size=b_size, epochs=max_epochs, shuffle=True, verbose=1)
print("Training finished \n")

Starting training 
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
19100/19100 [==============================] - 52s 3ms/step - loss: 0.0286 - categorical_accuracy: 0.0440
Epoch 2/20
19100/19100 [==============================] - 51s 3ms/step - loss: 0.0252 - categorical_accuracy: 0.2248
Epoch 3/20
19100/19100 [==============================] - 51s 3ms/step - loss: 0.0225 - categorical_accuracy: 0.2104
Epoch 4/20
19100/19100 [==============================] - 51s 3ms/step - loss: 0.0213 - categorical_accuracy: 0.1211
Epoch 5/20
19100/19100 [==============================] - 51s 3ms/step - loss: 0.0210 - categorical_accuracy: 0.0718
Epoch 6/20
19100/19100 [==============================] - 51s 3ms/step - loss: 0.0208 - categorical_accuracy: 0.0693
Epoch 7/20
19100/19100 [==============================] - 51s 3ms/step - loss: 0.0207 - categorical_accuracy: 0.0741
Epoch 8/20
19100/19100 [==============================] - 51s 3ms/step - loss: 0.0206 - categorical_accuracy: 0.07

In [29]:
p = textcnn.predict(test_x)

In [30]:
p[0],test_y[0]

(array([0.01147663, 0.13769454, 0.14181189, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.55124813, 0.4935875 , 0.53773236,
        0.520033  , 0.5210659 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [31]:
# 取出分数不为0的项
p_list = []
for k in p:
    p_tag = {}
    for i, score in enumerate(k.tolist()):
        if(score!=0):
            p_tag[score] = i
    p_list.append(p_tag)

In [32]:
def recall(true_tag, pre_tag):
    t = 0
    for i in pre_tag:
        if(i in true_tag):
            t += 1
    return t/len(true_tag)

def precision(true_tag, pre_tag):
    t = 0
    for i in pre_tag:
        if(i in true_tag):
            t += 1
    if len(pre_tag) == 0:
        return 1
    else:
        return t/len(pre_tag)

In [33]:
p_list_top5 = []
for i in range(len(p_list)):
    p_list_top5.append([p_list[i][k] for k in sorted(p_list[i].keys())][-5:])
    
p_list_top10 = []
for i in range(len(p_list)):
    p_list_top10.append([p_list[i][k] for k in sorted(p_list[i].keys())][-10:])
    
p_list_top116 = []
for i in range(len(p_list)):
    p_list_top116.append([p_list[i][k] for k in sorted(p_list[i].keys())][-116:])

In [34]:
true_tag_real = []
for row in test_y:
    true_tag_real.append(row.nonzero()[0])

In [35]:
recal_at_116_count = 0
recal_at_10_count = 0
recal_at_5_count = 0
precision_at_116_count = 0
precision_at_10_count = 0
precision_at_5_count = 0

for i in range(len(true_tag_real)):
    recal_at_10_count += recall(true_tag_real[i], p_list_top10[i])


for i in range(len(true_tag_real)):
    recal_at_5_count += recall(true_tag_real[i], p_list_top5[i])
    
for i in range(len(true_tag_real)):
    recal_at_116_count += recall(true_tag_real[i], p_list_top116[i])
    
for i in range(len(true_tag_real)):
    precision_at_10_count += precision(true_tag_real[i], p_list_top10[i])    

for i in range(len(true_tag_real)):
    precision_at_5_count += precision(true_tag_real[i], p_list_top5[i])
    
for i in range(len(true_tag_real)):
    precision_at_116_count += precision(true_tag_real[i], p_list_top116[i])
    
recall_at_116 = recal_at_116_count/len(true_tag_real)
recall_at_10 = recal_at_10_count/len(true_tag_real)
recall_at_5 = recal_at_5_count/len(true_tag_real)
precision_at_116 = precision_at_116_count/len(true_tag_real)
precision_at_10 = precision_at_10_count/len(true_tag_real)
precision_at_5 = precision_at_5_count/len(true_tag_real)

textcnn.summary()
print()
print("====================================")
print("Recall@116: {}".format(recall_at_116))
print("Precision@116: {}".format(precision_at_116_count/len(true_tag_real)))
print("F1-score@116: {}".format((2*recall_at_116*precision_at_10)/(recall_at_116+precision_at_10)))
print()
print("Recall@10: {}".format(recall_at_10))
print("Precision@10: {}".format(precision_at_10_count/len(true_tag_real)))
print("F1-score@10: {}".format((2*recall_at_10*precision_at_10)/(recall_at_10+precision_at_10)))
print()
print("Recall@5: {}".format(recall_at_5))
print("Precision@5: {}".format(precision_at_5_count/len(true_tag_real)))
print("F1-score@5: {}".format((2*recall_at_5*precision_at_5)/(recall_at_5+precision_at_5)))
print("====================================")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_seq (InputLayer)              (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 300, 64)      28590400    x_seq[0][0]                      
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 299, 100)     12900       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 298, 100)     19300       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_7 (

In [36]:
import datetime
textcnn.save("../model/textcnn-{}.h5".format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")))